In [1]:
import os
import sys

import warnings
warnings.filterwarnings("ignore") 

os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [38]:
# # boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\research\akadaner\2021-01-06 (5).xlsx")
# boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\data\sample_boiling_plan.xlsx")
# boilings = make_boilings_by_groups(boiling_plan_df)
# schedule = make_schedule(boilings, start_times={'water': '08:00', 'salt': '07:00'})
# frontend = make_frontend(schedule)
# # draw_excel_frontend(frontend, open_file=True)

In [41]:
df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\data\sample_boiling_plan.xlsx")
mark_consecutive_groups(df, 'boiling', 'boiling_group')
boiling_group_df = df[df['boiling_group'] == 2]
transformer = BoilingGroupToSchemaTransformer()
boilings_meltings, packings, melting_speed = transformer(boiling_group_df)
boilings_meltings, packings, melting_speed

([[[<BoilingFormFactor 3>, 26], [<BoilingFormFactor 13>, 974]],
  [[<BoilingFormFactor 4>, 1000]],
  [[<BoilingFormFactor 4>, 1000]]],
 {1: [[<SKU 12>, <BoilingFormFactor 3>, 26],
   [<SKU 25>, <BoilingFormFactor 13>, 974],
   [<SKU 30>, <BoilingFormFactor 4>, 2000]]},
 900)

In [9]:
# generate meltings by boilings
res = []
for boiling_meltings in boilings_meltings:
    boiling_volume = sum(x[1] for x in boiling_meltings)
    
    drenator = Container('Drenator', value=boiling_volume, max_pressures=[None, None])
    
    melting_processors = []
    cooling_processors = []
    for i, (bff, kg) in enumerate(boiling_meltings):
        melting_processor = Processor(f'Melting{i}', 
                                      items=['cheese_mass', bff],  
                                      max_pressures=[melting_speed, melting_speed], 
                                      processing_time=0,  
                                      limits=[kg, kg])
        melting_processors.append(melting_processor)
        
        cooling_processor = Processor(f'Cooling{i}', 
                                      items=[bff, bff], 
                                      max_pressures=[None, None], 
                                      processing_time=bff.post_processing_time / 60, 
                                      limits=[kg, kg])
        cooling_processors.append(cooling_processor)
    melting_queue = Queue('MeltingQueue', melting_processors, break_funcs={'in': lambda old, new: 1 / 12})
    cooling_queue = Queue('CoolingQueue', cooling_processors)
    
    res.append([drenator, melting_queue, cooling_queue])
res

[[<utils_ak.fluid_flow.actors.container.Container at 0x17c8979f430>,

In [10]:
# generate packing queues
packing_queues = []
for packing_team_id, grp in sorted(packings.items(), key=lambda v: v[0]):
    processors = []
    
    for j, (sku, bff, kg) in enumerate(grp):
        processor = Processor(f'Packing{j}', 
                              items=[bff, sku],
                             max_pressures=[sku.packing_speed, None],
                             processing_time=0,
                             limits=[kg, None])
        processors.append(processor)
    packing_queue = Queue(f'PackingQueue{packing_team_id}', processors, break_funcs={'in': lambda old, new: 1 / 12})
    packing_queues.append(packing_queue)
packing_queues

In [11]:
# for drenator, melting_queue, cooling_queue in res:
drenator, melting_queue, cooling_queue = res[0]

packing_hub = Hub('hub')

pipe_connect(drenator, melting_queue, 'drenator-melting')
pipe_connect(melting_queue, cooling_queue, 'melting-cooling')
pipe_connect(cooling_queue, packing_hub, 'cooling-hub')

for i, packing_queue in enumerate(packing_queues):
    pipe_connect(packing_hub, packing_queue, f'hub-packing_queue{i}')

flow = FluidFlow(drenator, verbose=True)
run_flow(flow)

maker, make = init_block_maker('root', axis=1)

for node in drenator.iterate('down'):
    if node.active_periods():
        for period in node.active_periods():
            label = '-'.join([str(node.id), str(period[0])])
            beg, end = period[1:]
            beg, end = custom_round(beg * 60, 5, 'ceil') // 5, custom_round(end * 60, 5, 'ceil') // 5
            make(label, x=[beg, 0], size=(end - beg, 1))
print(maker.root.tabular())


# # clean-up
# for node in drenator.iterate('down'):
#     node.reset()

# remove packers from current boiling
for packing_queue in packing_queues:
    pipe_disconnect(packing_hub, packing_queue)
    

Processing time 0
Flow:
    Container Drenator:default: 1000
    Queue: MeltingQueue: [["Melting0", [0, 0]], ["Melting1", [0, 0]]]
    Queue: CoolingQueue: [["Cooling0", [0, 0]], ["Cooling1", [0, 0]]]
    Queue: PackingQueue1: [["Packing0", [0, 0]], ["Packing1", [0, 0]], ["Packing2", [0, 0]]]

Processing time 0.028888888888888888
Flow:
    Container Drenator:default: 974.0
    Queue: MeltingQueue: [["Melting0", [0.0, 0.0]], ["Melting1", [0, 0]]]
    Queue: CoolingQueue: [["Cooling0", [26.0, 0]], ["Cooling1", [0, 0]]]
    Queue: PackingQueue1: [["Packing0", [0, 0]], ["Packing1", [0, 0]], ["Packing2", [0, 0]]]

Resuming 0.11222222222222222
Processing time 0.11222222222222222
Flow:
    Container Drenator:default: 974.0
    Queue: MeltingQueue: [["Melting0", [0.0, 0.0]], ["Melting1", [0, 0]]]
    Queue: CoolingQueue: [["Cooling0", [26.0, 0]], ["Cooling1", [0, 0]]]
    Queue: PackingQueue1: [["Packing0", [0, 0]], ["Packing1", [0, 0]], ["Packing2", [0, 0]]]

Processing time 1.111111111111111

In [12]:
melting_queue.active_periods()

[['cheese_mass', 0, 0.028888888888888888],
 ['cheese_mass', 0.11222222222222222, 1.1944444444444444]]

In [55]:
cooling_queue.active_periods()

[[<BoilingFormFactor 4>, 0, 2.361111111111111]]

In [15]:
packing_queue.active_periods('in')

[[<BoilingFormFactor 3>, 1.5, 1.5309523809523808],
 [<BoilingFormFactor 13>, 1.614285714285714, 2.6288690476190473]]

In [16]:
packing_queue.active_periods('out')

[[<SKU 12>, 1.5, 1.5309523809523808],
 [<SKU 25>, 1.614285714285714, 2.6288690476190473]]

In [32]:
df = pd.DataFrame(melting_queue.active_periods(), columns=['item', 'beg', 'end'])
df

,item,beg,end
0,cheese_mass,0.000000,0.028889
1,cheese_mass,0.112222,1.194444


In [33]:
# move to minutes
df['beg'] = df['beg'] * 60
df['end'] = df['end'] * 60
df

,item,beg,end
0,cheese_mass,0.000000,1.733333
1,cheese_mass,6.733333,71.666667


In [34]:
# round last end up
df.at[df.index[-1], 'end'] = custom_round(df.at[df.index[-1], 'end'], 5, 'ceil')
df

,item,beg,end
0,cheese_mass,0.000000,1.733333
1,cheese_mass,6.733333,75.000000


In [35]:
# round to five-minute intervals
df['beg'] = df['beg'].apply(lambda ts: None if ts is None else custom_round(ts, 5))
df['end'] = df['end'].apply(lambda ts: None if ts is None else custom_round(ts, 5))
df

,item,beg,end
0,cheese_mass,0.0,0.0
1,cheese_mass,5.0,75.0


In [36]:
# fix small intervals (like beg_ts and end_ts: 5, 5 -> 5, 10)
for i in range(len(df)):
    if i >= 1:
        while df.at[i, 'beg'] <= df.at[i - 1, 'end']:
            df.at[i, 'beg'] += 5

    while df.at[i, 'beg'] >= df.at[i, 'end']:
        df.at[i, 'end'] += 5

In [37]:
df

,item,beg,end
0,cheese_mass,0.0,5.0
1,cheese_mass,10.0,75.0


In [ ]:
with make('melting'):
    boiling_model = boiling_group.iloc[0]['boiling']

    serving = make('serving', size=(boiling_model.meltings.serving_time // 5, 0), push_func=add_push).block
    meltings = make('meltings', x=(serving.size[0], 0), push_func=add_push).block
    coolings = make('coolings', x=(serving.size[0], 0), push_func=add_push).block
    
    for 
    
    for i, (group, grp) in enumerate(boiling_plan.groupby('bff_group')):
        if i >= 1 and boiling_model.boiling_type == 'water':
            # non-first group - reconfigure time
            push(meltings, maker.create_block('melting_configuration', size=(1, 0)))

        # todo: del
        if boiling_model.boiling_type == 'salt':
            boiling_model.meltings.speed = 850 / 50 * 60

        melting_process = maker.create_block('melting_process',
                                             size=(int(custom_round(grp['kg'].sum() / boiling_model.meltings.speed * 60, 5, 'ceil')) // 5, 0),
                                             bff=grp.iloc[0]['bff'])

        push(meltings, melting_process)

        # add cooling
        cooling_process = maker.create_block('cooling_process', x=melting_process.props['x_rel'])
        _, cooling_make = init_block_maker(cooling_process)

        with cooling_make('start'):
            if boiling_model.boiling_type == 'water':
                cooling_make('cooling', size=(boiling_model.meltings.first_cooling_time // 5, 0))
                cooling_make('cooling', size=(boiling_model.meltings.second_cooling_time // 5, 0))
            elif boiling_model.boiling_type == 'salt':
                cooling_make('salting', size=(boiling_model.meltings.salting_time // 5, 0))

        with cooling_make('finish'):
            if boiling_model.boiling_type == 'water':
                cooling_make('cooling', size=(melting_process.size[0], 0))
            elif boiling_model.boiling_type == 'salt':
                cooling_make('salting', size=(melting_process.size[0], 0))

        push(coolings, cooling_process, push_func=add_push)

with make('packing',
          x=(listify(maker.root['melting']['coolings']['cooling_process'])[0]['start'].y[0], 0),
          packing_team_id=1, push_func=add_push):
    for i, (_, row) in enumerate(boiling_plan.iterrows()):
        sku, kg = row['sku'], row['kg']
        packing_speed = min(sku.packing_speed, boiling_model.meltings.speed)
        make('packing_process',
             size=[int(custom_round(kg / packing_speed * 60, 5, rounding='ceil')) // 5, 0],
             sku=sku)

        if i != len(boiling_plan) - 1:
            # add configuration
            conf_time_size = get_configuration_time(boiling_model, sku, boiling_plan.iloc[i + 1]['sku'])

            if conf_time_size:
                make('packing_configuration', size=[conf_time_size // 5, 0])

return maker.root


